In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras



/home/mirandalv/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mirandalv/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mirandalv/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mirandalv/anaconda3/lib/python3.6/site-packages/tensorflow/python/f

In [2]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
print(model.predict([10.0]))

[[18.978697]]


In [4]:
#https: // en.wikipedia.org / wiki / Huber_loss
"""
The first function does not work, because it returns a value;
The second function returns a tensor: https://www.kdnuggets.com/2018/05/wtf-tensor.html
"""

def my_huber_loss_self(y_pred, y_true):
    # hard code the threshold
    threshold = 1
    error = y_pred - y_true
    if tf.abs(error) <= threshold:
        loss = tf.square(error)/2.0
    else:
        loss = threshold*(tf.abs(error)-(0.5*threshold))
    return loss

def my_huber_loss(y_pred, y_true):
    # This is the standardized code from coursera, above function is how I implemented.
    threshold = 1
    error = y_pred - y_true
    is_small_error = tf.abs(error)<threshold
    small_error_loss = tf.square(error)/2
    big_error_loss = threshold * (tf.abs(error)-(0.5*threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)


# Above function has threshold hard coded, however, if we want to fine-tuning this parameter, we could use it as an
# input parameter by creating a rapper function.

# create a rapper
def my_huber_loss_with_threshold(threshold):
    def my_huber_loss(y_pred, y_true):
    # This is the standardized code from coursera, above function is how I implemented.
        threshold = 1
        error = y_pred - y_true
        is_small_error = tf.abs(error)<threshold
        small_error_loss = tf.square(error)/2
        big_error_loss = threshold * (tf.abs(error)-(0.5*threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    return my_huber_loss


# create a class for loss function
class MyHuberLoss(Loss):
    
    threshold = 1
    
    def __init__(self, threshold):
        super().__init__()
        self.threshold = threshold
    
    def call(self, y_pred, y_true):
        error = y_pred - y_true
        is_small_error = tf.abs(error)<=self.threshold
        small_error_loss = tf.square(error)/2
        big_error_loss = self.threshold*(tf.abs(error)-(0.5*self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)


In [7]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss)
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10.0]))

[[18.359705]]


In [ ]:
# use rapper
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(threshold=1))

# use class
model.compile(optimizer='sgd', loss=MyHuberLoss(threshold=1))
